In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from sklearn import metrics
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import defaultdict
import numpy as np
import pandas as pd

c:\Users\ZElion\anaconda3\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Charger le jeu de données 20newsgroups
data_20newsgroups = fetch_20newsgroups()

# Diviser le jeu de données en ensembles d'entraînement et de test
X_train, X_test = train_test_split(data_20newsgroups.data, test_size=0.2, random_state=42)

In [30]:
# Créer une représentation vectorielle des documents par CountVectorizer
vectorizer = CountVectorizer(max_features=1000, max_df=10, min_df=5, stop_words='english')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# Créer une représentation vectorielle des documents par TF-IDF features
vectorizer = TfidfVectorizer(max_df=0.5,min_df=5,stop_words="english",max_features=1000,)
X_train_vectorized = vectorizer.fit_transform(data_20newsgroups.data) 

In [ ]:
###################### Modèle LDA
lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(X_train_vectorized)

# Attribuer les documents de l'ensemble d'entraînement à des sujets
train_topic_assignments_lda = lda.transform(X_train_vectorized)

# Attribuer les documents de l'ensemble de test à des sujets
test_topic_assignments_lda = lda.transform(X_test_vectorized)

# Calculer le score de silhouette pour LDA
silhouette_lda = silhouette_score(test_topic_assignments_lda, test_topic_assignments_lda.argmax(axis=1))
print(f"Silhouette LDA Score: {silhouette_lda}")

# Utiliser le modèle IsolationForest pour évaluer la pertinence de l'entraînement sur les données de test
isolation_forest = IsolationForest(random_state=42)
isolation_forest.fit(test_topic_assignments_lda)
outliers_lda = isolation_forest.predict(test_topic_assignments_lda)
outlier_percentage_lda = list(outliers_lda).count(-1) / len(outliers_lda)
print(f"Outlier Percentage: {outlier_percentage_lda}")

topic_word_distributions_lda = lda.components_
topics = []
for topic_idx, topic_words in enumerate(topic_word_distributions_lda):
    top_words = [list(vectorizer.vocabulary_.keys())[i] for i in topic_words.argsort()[:-6:-1]]
    topics.append(', '.join(top_words))



In [31]:
############################ Modèle Kmean
kmeans_ = KMeans(n_clusters=20,max_iter=100,)

kmeans_.fit(X_train_vectorized)

# Attribuer les documents de l'ensemble d'entraînement à des sujets
train_topic_assignments_K = kmeans_.transform(X_train_vectorized)
print("Topics assignement Train :",train_topic_assignments_K,"\n")
print("Topics assignement Train Shape :",train_topic_assignments_K.shape)
# Attribuer les documents de l'ensemble de test à des sujets
test_topic_assignments_K = kmeans_.predict(X_test_vectorized)
print("Topics assignement Test :",test_topic_assignments_K.data)
print("Topics assignement Test Shape :",test_topic_assignments_K.shape)

# Calculer le score de silhouette pour KMeans
silhouette_kmean = silhouette_score(X_train_vectorized, kmeans_.labels_)
print(f"Silhouette Kmean Score: {silhouette_kmean}")

# Utiliser le modèle IsolationForest pour évaluer la pertinence de l'entraînement sur les données de test
isolation_forest = IsolationForest(random_state=42)
isolation_forest.fit(train_topic_assignments_K)
outliers_k = isolation_forest.predict(train_topic_assignments_K)
outlier_percentage_k = list(outliers_k).count(-1) / len(outliers_k)
print(f"Outlier Percentage: {outlier_percentage_k}")

topic_word_distributions_k = kmeans_.cluster_centers_
topics = []
for topic_idx, topic_words_k in enumerate(topic_word_distributions_k):
    top_words_k = [list(vectorizer.get_feature_names_out())[i] for i in topic_words_k.argsort()[:-6:-1]]
    topics.append(', '.join(top_words_k))

print("Topics :","\n",topics)
print("Top_W :","\n",top_words_k)
print("Labels KMeans :","\n",kmeans_.labels_)
print("Features KMeans :","\n",kmeans_.get_feature_names_out())
train_topic_assignments_K

c:\Users\ZElion\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Topics assignement Train : [[1.00208270e+00 3.42692282e+02 1.56211395e+02 ... 7.64329772e+01
  3.55246393e+01 2.13541565e+01]
 [3.29958639e+01 3.44276052e+02 1.59655880e+02 ... 8.32466216e+01
  4.84767986e+01 3.92937654e+01]
 [6.95294326e-02 3.42690823e+02 1.56208194e+02 ... 7.64264352e+01
  3.55105618e+01 2.13307290e+01]
 ...
 [6.95294326e-02 3.42690823e+02 1.56208194e+02 ... 7.64264352e+01
  3.55105618e+01 2.13307290e+01]
 [6.95294326e-02 3.42690823e+02 1.56208194e+02 ... 7.64264352e+01
  3.55105618e+01 2.13307290e+01]
 [6.95294326e-02 3.42690823e+02 1.56208194e+02 ... 7.64264352e+01
  3.55105618e+01 2.13307290e+01]] 

Topics assignement Train Shape : (9051, 20)
Topics assignement Test : <memory at 0x000001C384FEC040>
Topics assignement Test Shape : (2263,)
Silhouette Kmean Score: 0.8923140604238557
Outlier Percentage: 0.0883880234228262
Topics : 
 ['sumgait, mamma, nyr, har, vitamin', 'a86, 2di, 2tm, okz, bxn', 'w7, chz, 3dy, t7, tl', '75u, 2j, giz, a86, r8f', 'a86, 6um, 2di, 75u, 6

array([[1.00208270e+00, 3.42692282e+02, 1.56211395e+02, ...,
        7.64329772e+01, 3.55246393e+01, 2.13541565e+01],
       [3.29958639e+01, 3.44276052e+02, 1.59655880e+02, ...,
        8.32466216e+01, 4.84767986e+01, 3.92937654e+01],
       [6.95294326e-02, 3.42690823e+02, 1.56208194e+02, ...,
        7.64264352e+01, 3.55105618e+01, 2.13307290e+01],
       ...,
       [6.95294326e-02, 3.42690823e+02, 1.56208194e+02, ...,
        7.64264352e+01, 3.55105618e+01, 2.13307290e+01],
       [6.95294326e-02, 3.42690823e+02, 1.56208194e+02, ...,
        7.64264352e+01, 3.55105618e+01, 2.13307290e+01],
       [6.95294326e-02, 3.42690823e+02, 1.56208194e+02, ...,
        7.64264352e+01, 3.55105618e+01, 2.13307290e+01]])

In [ ]:
# Visualiser les clusters d'entraînement et les prédictions sur les données de test en 3D Statique
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Coordonnées des sujets d'entraînement
train_x = train_topic_assignments_lda[:, 0]
train_y = train_topic_assignments_lda[:, 1]
train_z = train_topic_assignments_lda[:, 2]
ax.scatter(train_x, train_y, train_z, c='blue', label='Train Clusters')

# Coordonnées des sujets de test
test_x = test_topic_assignments_lda[:, 0]
test_y = test_topic_assignments_lda[:, 1]
test_z = test_topic_assignments_lda[:, 2]
ax.scatter(test_x, test_y, test_z, c='red', label='Test Predictions')

ax.set_xlabel('Topic 1')
ax.set_ylabel('Topic 2')
ax.set_zlabel('Topic 3')
ax.legend()

plt.show()

In [ ]:
# Visualiser les clusters d'entraînement et les prédictions LDA sur les données de test en 3D Dynamique
# Créer les coordonnées x, y et z pour chaque point de données
x_train = train_topic_assignments_lda[:, 0]
y_train = train_topic_assignments_lda[:, 1]
z_train = train_topic_assignments_lda[:, 2]

x_test = test_topic_assignments_lda[:, 0]
y_test = test_topic_assignments_lda[:, 1]
z_test = test_topic_assignments_lda[:, 2]

# Afficher les clusters trouvés lors de l'entraînement en 3D
lda_labels = KMeans(n_clusters=10, random_state=0).fit_predict(train_topic_assignments_lda)

fig = go.Figure(data=[
    go.Scatter3d(x=x_train, y=y_train, z=z_train, mode='markers', marker=dict(size=2, opacity=0.7, color=lda_labels),name='Train'),
    go.Scatter3d(x=x_test, y=y_test, z=z_test, mode='markers', marker=dict(size=2, opacity=0.7, color=lda_labels),name='Test')
])
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

fig.show()


In [ ]:
# Visualiser les clusters d'entraînement et les prédictions KMeans sur les données de test en 3D Dynamique
# Créer les coordonnées x, y et z pour chaque point de données
x_train = train_topic_assignments_K[:, 0]
y_train = train_topic_assignments_K[:, 1]
z_train = train_topic_assignments_K[:, 2]

x_test = test_topic_assignments_K[:, 0]
y_test = test_topic_assignments_K[:, 1]
z_test = test_topic_assignments_K[:, 2]

# Afficher les clusters trouvés lors de l'entraînement en 3D
cluster_labels = KMeans(n_clusters=10, random_state=0).fit_predict(train_topic_assignments_K)

fig = go.Figure(data=[
    go.Scatter3d(x=x_train, y=y_train, z=z_train, mode='markers', marker=dict(size=2, opacity=0.7, color=cluster_labels),name='Train'),
    go.Scatter3d(x=x_test, y=y_test, z=z_test, mode='markers', marker=dict(size=2, opacity=0.7, color=cluster_labels),name='Test')
])
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
# fig.(*cluster_labels.item(), title='Clusters')
fig.show()


In [ ]:
dataset = fetch_20newsgroups()
df = pd.DataFrame()
df["data"] = dataset["data"]
df["target"] = dataset["target"]
df["target_names"] = df.target.apply(lambda row: dataset["target_names"][row])
df.head()

def clean_text(text):
    return " ".join([ Word(word).lemmatize() for word in re.sub("[^A-Za-z0-9]+", " ", text).lower().split() if word not in stopword])    

df["data_str"] = df.data.apply(lambda row: clean_text(row) )

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

perp_components = defaultdict(dict)
for i in [10,30,50,100]:
    for ngram in [(1,1),(1,2),(1,3),(2,2),(3,3)]:
        vectorizer = TfidfVectorizer(ngram_range=ngram, max_features= 2**10)
        text_to_vector = vectorizer.fit_transform(data.data_str.values)
        print("Ngram ",ngram )
        print("Perplexity ", i)
        X_embedded = TSNE(init="random",perplexity=i ).fit_transform(text_to_vector)
        ngram_str = str(ngram[0])+"_"+str(ngram[1])
        perp_components[i][ngram_str] = X_embedded
        # sns settings
        sns.set(rc={'figure.figsize':(15,15)})
        # colors
        palette = sns.color_palette("hls", len(set(df.target_names.values.tolist())))        
        y = df.target_names.values.tolist()
        # plot
        sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, legend='full', palette=palette)
        title = "t-SNE- 20News group - TfIdf - "+ngram_str+"- tSNE perplexity - "+str(i)
        plt.savefig(title)
        plt.title(title)
        plt.show()